In [1]:
import torch
import numpy as np
from torch import nn

import warnings

warnings.filterwarnings("ignore")

In [2]:
# get sample data

import polars as pl
import yfinance as yf
import re

prices = yf.download("SPLG", start='2023-01-01', end='2024-01-01')

df = (
    pl
    .from_pandas(
        prices
        .reset_index()
    ).with_columns(
        pl.lit("SPLG").alias("Ticker")
    )
)

df.columns = [re.sub(r"[^\w\s]","",header.split(",")[0]) for header in df.columns]

df.head()

[*********************100%***********************]  1 of 1 completed


Date,Close,High,Low,Open,Volume,Ticker
datetime[ns],f64,f64,f64,f64,i64,str
2023-01-03 00:00:00,43.092079,43.71758,42.755273,43.486625,3688300,"""SPLG"""
2023-01-04 00:00:00,43.42889,43.659842,42.995852,43.342283,4335600,"""SPLG"""
2023-01-05 00:00:00,42.92849,43.197934,42.861129,43.188313,4449300,"""SPLG"""
2023-01-06 00:00:00,43.900417,44.03514,42.928489,43.284542,2160500,"""SPLG"""
2023-01-09 00:00:00,43.881172,44.545166,43.861929,44.15062,4251700,"""SPLG"""


# SPLG Simple RNN Class

In [10]:
# Extremely basic 1-d 1-param Elman RNN cell - with another linear transformation after tanh activation

class SPLGRNN(nn.Module):
    def __init__(self, sequence_length = 1):
        super(SPLGRNN, self).__init__()

        self.w_x = nn.Parameter(
            torch.randn(1,1, requires_grad=True, dtype=torch.float32)
        )
        self.w_h = nn.Parameter(
            torch.randn(1,1, requires_grad=True, dtype=torch.float32)
        )
        self.b_h = nn.Parameter(
            torch.randn(1, requires_grad=True, dtype=torch.float32)
        )

        self.w_y = nn.Parameter(
            torch.randn(1, requires_grad=True, dtype=torch.float32)
        )
        self.b_y = nn.Parameter(
            torch.randn(1, requires_grad=True, dtype=torch.float32)
        )

        self.seq_len = sequence_length

    def forward(self, x, h = None):
        """
        Inputs:

        x = input data
        h = hidden state value from previous iteration (default to 0 if not applicable)
        """

        if len(x.shape) <= 1:
            x = x.unsqueeze(1)
            output = []

        if h is None:
            h = torch.zeros(1, dtype=torch.float32)

        h_1 = h

        x = x.to(dtype=torch.float32) #ensure type is aligned
        seq = 0
        
        for entry in x: #loop is used to ensure hidden states carry through iterations
            h_1 = torch.relu(
                entry @ self.w_x.t() + h_1 @ self.w_h.t() + self.b_h
            )
            y = h_1 @ self.w_y.t() + self.b_y
            output.append(y)
            if seq >= self.seq_len:
                seq = 0
                h_1 = h
            else:
                seq += 1

        output = torch.stack(output)
        if len(x.shape) <= 1:
            output = output.squeeze(1)

        return output, h_1


## Training

### Prep dataset

In [15]:
x_train = torch.FloatTensor(df["Close"].to_list()[:-1])
y_train = torch.FloatTensor(df["Close"].to_list()[1:])

print(x_train.shape)
print(y_train.shape)

torch.Size([249])
torch.Size([249])


### Setup Training Loop

In [16]:
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

# Setup hyperparamters
sequence_length = 5
batch_size = 50
epochs = 1000
learning_rate = 0.01

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Setup model
model = SPLGRNN(sequence_length=sequence_length)

# Setup loss function and optimizer
criterion = nn.MSELoss() #there is no RMSE
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1, epochs + 1):
    total_loss = 0
    model.train()
    for batch_id, (data, target) in enumerate(train_loader): #for each batch, also get the index of batch
        optimizer.zero_grad()
        output, _ = model(data) #forward pass
        loss = criterion(output, target)
        loss.backward() #compute gradients
        optimizer.step() #update weights
        
        total_loss += loss.item()

        if batch_id % 100 == 0 and epoch % 100 == 0: #update on training iterations
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id * len(data), len(train_loader.dataset),
                100. * batch_id / len(train_loader), loss.item()))


Train Epoch: 100 [0/249 (0%)]	Loss: 16.223093
Train Epoch: 200 [0/249 (0%)]	Loss: 16.495329
Train Epoch: 300 [0/249 (0%)]	Loss: 16.286247
Train Epoch: 400 [0/249 (0%)]	Loss: 15.068651
Train Epoch: 500 [0/249 (0%)]	Loss: 16.254644
Train Epoch: 600 [0/249 (0%)]	Loss: 13.322811
Train Epoch: 700 [0/249 (0%)]	Loss: 15.926310
Train Epoch: 800 [0/249 (0%)]	Loss: 10.887211
Train Epoch: 900 [0/249 (0%)]	Loss: 11.582383
Train Epoch: 1000 [0/249 (0%)]	Loss: 8.767213


In [17]:
y = model(x_train)
y

(tensor([[45.9542],
         [46.2629],
         [46.0607],
         [46.4551],
         [46.4490],
         [46.5781],
         [46.6312],
         [46.8724],
         [46.9479],
         [46.9051],
         [46.6271],
         [46.4850],
         [46.6508],
         [47.0369],
         [47.0269],
         [47.0307],
         [47.2342],
         [47.2821],
         [46.8660],
         [47.3118],
         [47.5133],
         [47.7803],
         [47.5898],
         [47.4754],
         [47.5430],
         [47.5104],
         [47.3459],
         [47.3804],
         [47.5997],
         [47.6046],
         [47.4843],
         [47.3928],
         [47.3454],
         [46.9695],
         [46.9405],
         [47.0421],
         [46.6743],
         [46.9078],
         [46.8424],
         [46.7716],
         [46.9083],
         [47.2063],
         [47.0343],
         [46.9329],
         [46.9638],
         [46.6195],
         [46.3676],
         [46.3312],
         [46.4551],
         [46.5195],


In [18]:
model._parameters

{'w_x': Parameter containing:
 tensor([[0.1684]], requires_grad=True),
 'w_h': Parameter containing:
 tensor([[0.0044]], requires_grad=True),
 'b_h': Parameter containing:
 tensor([8.7976], requires_grad=True),
 'w_y': Parameter containing:
 tensor([2.4152], requires_grad=True),
 'b_y': Parameter containing:
 tensor([7.1824], requires_grad=True)}